# Imports

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import sys
###
#!pip install openslide-python
#https://github.com/deroneriksson/python-wsi-preprocessing/
###
sys.path.append('../python-wsi-preprocessing/')
from deephistopath.wsi import slide, filter, tiles, util

import sys
sys.path.append('../fastai/') #fastai version 1.0
from fastai.vision import *
from fastai.core import parallel

import pandas as pd
import numpy as np
import os
from pathlib import Path
from functools import partial, update_wrapper
from tqdm import tqdm_notebook as tqdm
from enum import Enum
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

PATH = Path('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/')
FONT_PATH=PATH/'1984-Happines-Regular.ttf'
ROIS_CORTICOTROP = PATH/'rois_corticotrop'
ROIS_CORTICOTROP_FILTERED = PATH/'rois_corticotrop_filtered'
ROIS_GONADOTROP = PATH/'rois_gonadotrop'
ROIS_GONADOTROP_FILTERED = PATH/'rois_gonadotrop_filtered'
TILES_CORTICOTROP = PATH/'tiles_corticotrop'
TILES_GONADOTROP = PATH/'tiles_gonadotrop'
slide.BASE_DIR = PATH
#slide.TRAIN_PREFIX = 'hypophysen'
#slide.SRC_TRAIN_DIR = ROIS_CORTICOTROP
slide.SRC_TRAIN_DIR = ROIS_GONADOTROP

def show(np):
    return util.np_to_pil(np)

Path.ls = lambda x: [p for p in list(x.iterdir()) if '.ipynb_checkpoints' not in p.name]

def show_multiple_images(path, rows = 3, figsize=(128, 64)):
    imgs = [open_image(p) for p in path.ls()]
    show_all(imgs=imgs, r=rows, figsize=figsize)
    
def show_multiple_images_big(path:pathlib.Path):
    for p in path.ls():
        plt.imshow(mpimg.imread(str(p)))
        plt.show()
        
class AdenomaType(Enum):
    corticotrop = 1
    gonadotrop = 2

In [ ]:
adenomaType = AdenomaType.gonadotrop

if adenomaType == AdenomaType.gonadotrop:
    rois_filtered_path = ROIS_GONADOTROP_FILTERED
    rois_path = ROIS_GONADOTROP
    tiles_path = TILES_GONADOTROP
    
if adenomaType == AdenomaType.corticotrop:
    rois_filtered_path = ROIS_CORTICOTROP_FILTERED
    rois_path = ROIS_CORTICOTROP
    tiles_path = TILES_CORTICOTROP

# Overwrite parts of wsi lib

In [ ]:
slide.SCALE_FACTOR = 1
slide.BASE_DIR = PATH/'data'
tiles.SUMMARY_TITLE_FONT_PATH = str(FONT_PATH)
tiles.FONT_PATH = str(FONT_PATH)
tiles.DISPLAY_TILE_SUMMARY_LABELS = True
tiles.LABEL_ALL_TILES_IN_TOP_TILE_SUMMARY = True
tiles.BORDER_ALL_TILES_IN_TOP_TILE_SUMMARY = True
tiles.TISSUE_LOW_THRESH = 20

##################################################################################################################

slide.open_slide = slide.open_image

##################################################################################################################

def get_image_path(folder_path, slide_num):
    return folder_path.ls()[slide_num]

slide.get_filter_image_result = partial(get_image_path, rois_filtered_path)
slide.get_training_image_path = partial(get_image_path, rois_path)
slide.get_training_slide_path = partial(get_image_path, rois_path)

##################################################################################################################

def parse_dimensions_from_image_filename(img_path):
    shape = slide.open_image_np(img_path).shape
    return shape[1], shape[0], shape[1], shape[0]

slide.parse_dimensions_from_image_filename = parse_dimensions_from_image_filename

##################################################################################################################

def tile_to_pil_tile(tile):
  """
  Convert tile information into the corresponding tile as a PIL image read from the whole-slide image file.

  Args:
    tile: Tile object.

  Return:
    Tile as a PIL image.
  """
  t = tile
  filepath = slide.get_training_image_path(t.slide_num)
  img = slide.open_image_np(filepath)
  x, y = t.o_c_s, t.o_r_s
  w, h = t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s
  tile = img[t.o_r_s:t.o_r_e, t.o_c_s:t.o_c_e,:]
  tile_pil = util.np_to_pil(tile)
  return tile_pil

tiles.tile_to_pil_tile = tile_to_pil_tile

##################################################################################################################
def get_tile_image_path(tile):
  """
  Obtain tile image path based on tile information such as row, column, row pixel position, column pixel position,
  pixel width, and pixel height.

  Args:
    tile: Tile object.

  Returns:
    Path to image tile.
  """
  t = tile
  roi_name = slide.get_training_image_path(t.slide_num).stem
  tile_path = os.path.join(PATH, tiles_path,
                           roi_name + "-" + 'tile' + "-r%d-c%d-x%d-y%d-w%d-h%d" % (
                             t.r, t.c, t.o_c_s, t.o_r_s, t.o_c_e - t.o_c_s, t.o_r_e - t.o_r_s) + "." + 'png')
  return tile_path

slide.get_tile_image_path = get_tile_image_path

##################################################################################################################

def top_tiles(self):
    """
    Retrieve the top-scoring tiles.

    Returns:
       List of the top-scoring tiles.
    """
    sorted_tiles = self.tiles_by_score()
    top_tiles = [tile for tile in sorted_tiles
                 if check_tile(tile)]
    print(f'Number of top tiles/all tiles: {len(top_tiles)}/{len(sorted_tiles)}')
    return top_tiles

def check_tile(tile):
    width = tile.o_c_e - tile.o_c_s
    height = tile.o_r_e - tile.o_r_s
    return tile.score > 0.45 and width >= 0.7*tiles.COL_TILE_SIZE and height >= 0.7*tiles.ROW_TILE_SIZE


tiles.TileSummary.top_tiles = top_tiles

##################################################################################################################
from deephistopath.wsi.tiles import (hsv_saturation_and_value_factor, 
                                     hsv_purple_pink_factor, 
                                     tissue_quantity_factor, 
                                     tissue_quantity)

def score_tile(np_tile, tissue_percent, slide_num, row, col):
  """
  Score tile based on tissue percentage, color factor, saturation/value factor, and tissue quantity factor.

  Args:
    np_tile: Tile as NumPy array.
    tissue_percent: The percentage of the tile judged to be tissue.
    slide_num: Slide number.
    row: Tile row.
    col: Tile column.

  Returns tuple consisting of score, color factor, saturation/value factor, and tissue quantity factor.
  """
  color_factor = hsv_purple_pink_factor(np_tile)
  s_and_v_factor = hsv_saturation_and_value_factor(np_tile)
  amount = tissue_quantity(tissue_percent)
  quantity_factor = tissue_quantity_factor(amount)
  combined_factor = color_factor * s_and_v_factor
  score = tissue_percent * combined_factor / 1000.0
  # scale score to between 0 and 1
  score = 1.0 - (10.0 / (10.0 + score))
  print(f'tp: {tissue_percent}')
  print(f'cf: {combined_factor}')
  print(f'score: {score}')  
  return score, color_factor, s_and_v_factor, quantity_factor

tiles.score_tile = score_tile
#tiles.score_tile = tiles.score_tile
#############################################################################################################

# Filter images

In [ ]:
def filter_roi(img_path:pathlib.Path, index:int):
    if img_path.suffix == '.png':
        try:
            img_pil = slide.open_image(img_path)
            img_np = util.pil_to_np_rgb(img_pil)
            grayscale_np = filter.filter_rgb_to_grayscale(img_np)
            complement_np = filter.filter_complement(grayscale_np)
            otsu_np = filter.filter_otsu_threshold(complement_np).astype(np.bool)
            filtered_img_np = util.mask_rgb(img_np, otsu_np)
            filtered_img_pil = util.np_to_pil(filtered_img_np)
            filtered_path = rois_filtered_path/f'{img_path.stem}-filtered{img_path.suffix}'
            filtered_img_pil.save(filtered_path)
        except:
            print(img_path)

parallel(filter_roi, rois_path.ls(), max_workers=25)

In [ ]:
show_multiple_images_big(rois_filtered_path)

# Create tiles

## multi process

In [ ]:
tiles.multiprocess_filtered_images_to_tiles(display=False, 
                                            save_summary=False, 
                                            save_data=False, 
                                            save_top_tiles=True,
                                            html=True, 
                                            image_num_list=list(range(0, len(rois_filtered_path.ls()))))

In [ ]:
show_multiple_images_big(tiles_path)

In [8]:
#!rm -r {rois_path/'*'}
#!rm -r {rois_filtered_path/'*'}
#!rm -r {tiles_path/'*'}

## single process

In [ ]:
#!rm -r {tiles_path/'*'}
for n, p in tqdm(enumerate(rois_filtered_path.ls()), total=len(rois_filtered_path.ls())-1):
    if p.suffix == '.png':
        tiles.summary_and_tiles(n, display=False, save_summary=False, save_data=False, save_top_tiles=True)

In [7]:
p1 = Path('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop/815-16-III-HE-ROI_13-LH+FSH.png')
p2 = Path('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop/815-16-III-HE-ROI_14-LH+FSH.png')
p3 = Path('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop/203-16-Z-HE-ROI_10-FSH.png')

failed=[]

for n, p in tqdm(enumerate([p1,p2,p3]), total=3):
    if p.suffix == '.png':
        try:
            tiles.summary_and_tiles(n, display=False, save_summary=False, save_data=False, save_top_tiles=True)
        except:
            failed.append(p)
print(failed)

RGB                  | Time: 0:00:01.775846  Type: uint8   Shape: (5680, 9544, 3)
RGB                  | Time: 0:00:01.775228  Type: uint8   Shape: (5680, 9544, 3)
tp: 80.06868362426758
cf: 1180.402679342409
score: 0.9043183873921133
tp: 77.86703109741211
cf: 1203.297300533999
score: 0.903565369881293
tp: 70.31087875366211
cf: 1436.8855255712522
score: 0.9099331844500471
tp: 67.4433708190918
cf: 1388.3731799275602
score: 0.9035089614670443
tp: 69.6537971496582
cf: 1307.6250673505733
score: 0.9010694894605153
tp: 61.50951385498047
cf: 1401.679997362587
score: 0.8960678899371382
tp: 68.53761672973633
cf: 1570.5560047540844
score: 0.9149964644313339
tp: 69.26813125610352
cf: 1722.3590573555718
score: 0.9226632268176608
tp: 30.746841430664062
cf: 2660.2773678351587
score: 0.8910617546571132
tp: 17.451290967987802
cf: 2562.642284469563
score: 0.817256191396559
tp: 78.81250381469727
cf: 1095.6448592236472
score: 0.8962122824499184
tp: 79.22353744506836
cf: 1188.047099042175
score: 0.90395816

In [6]:
failed

[PosixPath('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop_filtered/815-16-III-HE-ROI_13-LH+FSH-filtered.png'),
 PosixPath('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop_filtered/815-16-III-HE-ROI_14-LH+FSH-filtered.png'),
 PosixPath('/home/Deep_Learner/work/network/datasets/Hypophysenadenome/rois_gonadotrop_filtered/203-16-Z-HE-ROI_10-FSH-filtered.png')]